<a href="https://colab.research.google.com/github/keudithmanco-gif/Proyecto-IA/blob/main/99_Modelo_soluci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ============================
# Importar librerias
# ============================
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
import os

# Opcional para Colab file download/upload
try:
    from google.colab import files
except Exception:
    files = None


In [2]:
#Cargar recursos y preparar entorno
#archivo de inicialización desde el curso.

!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()


replicating local resources


In [3]:
#Se carga el archivo kaggle.json personal para autenticarse
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"keudithmancosencio","key":"be0eebf9bedbde9e2c7805112fb57647"}'}

In [4]:
#Configuración de permisos de kaggle

import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia


  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.47GB/s]


In [5]:
!unzip udea*.zip > /dev/null && echo "✅ Descompresión finalizada"


✅ Descompresión finalizada


In [6]:
!ls -lh *.csv
#Verificando si ya están descomprimidos los archivos en el entorno de colab


-rw-r--r-- 1 root root 4.5M Sep 16 01:46 submission_example.csv
-rw-r--r-- 1 root root  57M Sep 16 01:46 test.csv
-rw-r--r-- 1 root root 138M Sep 16 01:46 train.csv


In [7]:
#Identificar la cantidad de datos de cada archivo
!wc *.csv

   296787    296787   4716673 submission_example.csv
   296787   4565553  59185238 test.csv
   692501  10666231 143732437 train.csv
  1286075  15528571 207634348 total


In [8]:
# ===============================================================
#                 CARGA DE ARCHIVOS
# ===============================================================
# Define the paths for the datasets
TRAIN_PATH = "train.csv"
TEST_PATH  = "test.csv"

train_df = pd.read_csv(TRAIN_PATH)
test_df  = pd.read_csv(TEST_PATH)

In [9]:
train_df.columns


Index(['ID', 'PERIODO_ACADEMICO', 'E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO',
       'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA',
       'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE',
       'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PRIVADO_LIBERTAD',
       'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_TIENEINTERNET.1',
       'F_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL', 'INDICADOR_1', 'INDICADOR_2',
       'INDICADOR_3', 'INDICADOR_4'],
      dtype='object')

In [10]:
#Se selecciona la columna que se quiere predecir (RENDIMIENTO_GLOBAL) y se separan las variables predictoras (X) de la variable objetivo (y).
# ===============================================================
#                 TARGET
# ===============================================================
target = "RENDIMIENTO_GLOBAL"

if target not in train_df.columns:
    raise Exception(f"La variable objetivo '{target}' NO está en train.csv")

X = train_df.drop(columns=[target])
y = train_df[target]

#Como LightGBM trabaja internamente con números, se convierten las etiquetas de texto en valores numéricos.
# ===============================================================
#     MAPEO DE CLASES
# ===============================================================
label_map = {
    "bajo": 0,
    "medio-bajo": 1,
    "medio-alto": 2,
    "alto": 3
}
y = y.map(label_map)

#Identifica automáticamente qué columnas son categóricas y cuáles son numéricas para aplicar diferentes preprocesos a cada tipo.
# ===============================================================
#     COLUMNAS CATEGÓRICAS Y NUMÉRICAS
# ===============================================================
cat_cols = X.select_dtypes(include=["object"]).columns
num_cols = X.select_dtypes(exclude=["object"]).columns

print("\nColumnas categóricas detectadas:", list(cat_cols))
print("Columnas numéricas detectadas:", list(num_cols))

#Se aplica:
#One-Hot Encoding a columnas categóricas
#Passthrough (dejarlas tal cual) a las numéricas
#Esto prepara los datos para que el modelo pueda interpretarlos correctamente.

# ===============================================================
#     PREPROCESAMIENTO (One-Hot Encoding)
# ===============================================================
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", "passthrough", num_cols),
    ]
)

#Se crea el modelo LightGBM, un algoritmo muy eficiente para clasificación, optimizado con varios hiperparámetros (n_estimators, learning_rate, num_leaves, etc.).
# ===============================================================
#     MODELO LIGHTGBM
# ===============================================================
from lightgbm import LGBMClassifier

model = LGBMClassifier(
    n_estimators=1200,
    max_depth=-1,              # -1 = sin límite
    learning_rate=0.03,
    num_leaves=50,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="multiclass",
    class_weight=None,
    random_state=42,
    n_jobs=-1,
    metric="multi_logloss",
)

#El pipeline integra:
#1. El preprocesamiento y
#2. El modelo
#Esto garantiza que el mismo flujo se aplique tanto al entrenamiento como a las predicciones.

# ===============================================================
#     PIPELINE
# ===============================================================
clf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", model)
])

#Se separan los datos en:
#80% entrenamiento
#20% validación
#Usando stratify para mantener las proporciones de clases.

# ===============================================================
#     TRAIN / VALIDACIÓN
# ===============================================================
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

#Entrenar el pipeline con los datos procesados.
# ===============================================================
#     ENTRENAMIENTO
# ===============================================================
print("\nEntrenando modelo LightGBM...")
clf.fit(X_train, y_train)

#Calcula el accuracy del modelo en el conjunto de validación.
# ===============================================================
#     EVALUACIÓN
# ===============================================================
val_acc = clf.score(X_val, y_val)
print(f"\nAccuracy en validación con LightGBM: {val_acc:.4f}")

#Se generan las predicciones para los datos de test.
# ===============================================================
#     PREDICCIONES SOBRE TEST
# ===============================================================
y_pred = clf.predict(test_df)

#Las predicciones numéricas del modelo se transforman nuevamente a palabras:
#0 → bajo, 1 → medio-bajo, etc.
# Convertir números → palabras
inverse_label_map = {v: k for k, v in label_map.items()}
y_pred_labels = [inverse_label_map[i] for i in y_pred]



Columnas categóricas detectadas: ['E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO', 'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA', 'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PRIVADO_LIBERTAD', 'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_TIENEINTERNET.1', 'F_EDUCACIONMADRE']
Columnas numéricas detectadas: ['ID', 'PERIODO_ACADEMICO', 'INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']

Entrenando modelo LightGBM...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.073340 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2886
[LightGBM] [Info] Number of data points in the train set: 554000, number of used features: 868
[LightGBM] [Info] Start training from score -1.387089
[LightGBM] [Info] Start training from score -1.391216
[LightGBM] [Info] Start training from score -1

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



Accuracy en validación con LightGBM: 0.4380


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [11]:
# ============================================================
#   CREAR DATAFRAME FINAL DE ENVÍO
# ============================================================

submission = pd.DataFrame({
    "ID": test_df['ID'],
    "RENDIMIENTO_GLOBAL": y_pred_labels
})

# Mostrar preview
display(submission.head())

# Guardar archivo
submission.to_csv("submission.csv", index=False)
print("Archivo 'submission.csv' generado correctamente.")


# ============================================================
#   ENVIAR A KAGGLE
# ============================================================

!kaggle competitions submit \
    -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia \
    -f submission.csv \
    -m "Submit_predicción_Gabi"

,ID,RENDIMIENTO_GLOBAL
0,550236,bajo
1,98545,medio-alto
2,499179,alto
3,782980,bajo
4,785185,bajo


Archivo 'submission.csv' generado correctamente.
100% 4.05M/4.05M [00:00<00:00, 5.73MB/s]
Successfully submitted to UDEA/ai4eng 20252 - Pruebas Saber Pro Colombia